In [13]:
import pandas as pd
df = pd.read_parquet('../data/processed/index_100k.parquet')
print(df.head(10))

          id           submitter  \
0  0704.1020     Gyorgy Ottucsak   
1  0704.1028       Jianlin Cheng   
2  0704.1274      Dev Rajnarayan   
3  0704.1394  Tarik Had\v{z}i\'c   
4  0704.1409       Yao Hengshuai   
5  0704.1675     Kristina Lerman   
6  0704.2010   Juliana Bernardes   
7  0704.2092         Jinsong Tan   
8  0704.2668        Alex Smola J   
9  0704.3157   Giorgio Terracina   

                                             authors  \
0  Andras Gyorgy, Tamas Linder, Gabor Lugosi, Gyo...   
1                                      Jianlin Cheng   
2             David H. Wolpert and Dev G. Rajnarayan   
3  Tarik Hadzic, Rune Moller Jensen, Henrik Reif ...   
4                                      Yao HengShuai   
5           Anon Plangprasopchok and Kristina Lerman   
6  Juliana S Bernardes, Alberto Davila, Vitor San...   
7                                        Jinsong Tan   
8  Le Song, Alex Smola, Arthur Gretton, Karsten B...   
9  Giorgio Terracina, Nicola Leone, Vincenz

In [15]:
temp_df = df["primary_category"].value_counts()
temp_df

primary_category
cs.LG    70000
cs.AI    30000
Name: count, dtype: int64

In [10]:
import json

filepath = "../data/raw/arxiv-metadata-oai-snapshot.json"
count = 0

with open(filepath, 'r') as f:
    for line in f:
        paper = json.loads(line)
        primary_cat = paper['categories'].split()[0]
        if primary_cat == "cs.AI":
            count += 1

print(f"Total cs.AI papers: {count}")

Total cs.AI papers: 151253


In [16]:
# Quick script: eval_foreign_model.py
import numpy as np
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer

PROJECT_ROOT = Path(__file__).parent if '__file__' in dir() else Path('.')

# Load holdout data
holdout_df = pd.read_parquet("../data/processed/holdout_papers.parquet")
print(f"Loaded {len(holdout_df)} holdout papers")

# Load foreign model
model = SentenceTransformer("../models/foreign_finetuned_st")
print(f"Loaded model from foreign_finetuned_st")

# Generate embeddings
print("Generating embeddings...")
embeddings = model.encode(holdout_df['text'].tolist(), show_progress_bar=True)

# Save to where evaluate_holdout.py looks
output_path = Path("models/foreign_st/holdout_embeddings.npy")
output_path.parent.mkdir(parents=True, exist_ok=True)
np.save(output_path, embeddings)
print(f"Saved embeddings to {output_path}")
print(f"Shape: {embeddings.shape}")

Loaded 5000 holdout papers
Loaded model from foreign_finetuned_st
Generating embeddings...


Batches:   0%|          | 0/157 [00:00<?, ?it/s]

Saved embeddings to models/foreign_st/holdout_embeddings.npy
Shape: (5000, 384)


In [18]:
df = pd.read_csv('../data/annotated/hard_pairs.csv')
print(f"Score range: {df['ada_sim'].min():.3f} - {df['ada_sim'].max():.3f}")

Score range: 0.597 - 0.995
